In [167]:
import pandas as pd
import numpy as np
import os
import email
import email.policy
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords, words
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from string import punctuation
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import heapq

In [168]:
os.listdir('./Database')

['ham', 'spam']

In [169]:
arquivosNormais = [nomeArquivo for nomeArquivo in (os.listdir("./Database/ham")) if len(nomeArquivo) > 20]
arquivosSpam = [nomeArquivo for nomeArquivo in (os.listdir("./Database/spam")) if len(nomeArquivo) > 20]

emailsNome = arquivosNormais + arquivosSpam

In [170]:
def carregarEmail(tipo, arquivo):
    caminho = ""

    if tipo == "spam":
        caminho = "./Database/spam"
    if tipo == "ham":
        caminho = "./Database/ham"

    with open(f"{caminho}/{arquivo}", "rb") as file:
        return email.parser.BytesParser(policy=email.policy.default).parse(file)


emailsNormais = [carregarEmail("ham", arquivo) for arquivo in arquivosNormais]
emailsSpam = [carregarEmail("spam", arquivo) for arquivo in arquivosSpam]

emails = emailsNormais + emailsSpam

aux = emailsNormais[0]

In [171]:
for i in aux.keys():
    print(f"{i} : {aux[i]}")

print('\n')
print('Conteudo:', aux.get_content())

Return-Path : <exmh-workers-admin@example.com>
Delivered-To : zzzz@localhost.netnoteinc.com
Received : from localhost (localhost [127.0.0.1])	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id D03E543C36	for <zzzz@localhost>; Thu, 22 Aug 2002 07:36:16 -0400 (EDT)
Received : from localhost (localhost [127.0.0.1])	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id D03E543C36	for <zzzz@localhost>; Thu, 22 Aug 2002 07:36:16 -0400 (EDT)
Received : from localhost (localhost [127.0.0.1])	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id D03E543C36	for <zzzz@localhost>; Thu, 22 Aug 2002 07:36:16 -0400 (EDT)
Received : from localhost (localhost [127.0.0.1])	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id D03E543C36	for <zzzz@localhost>; Thu, 22 Aug 2002 07:36:16 -0400 (EDT)
Delivered-To : zzzz@localhost.netnoteinc.com
Received : from localhost (localhost [127.0.0.1])	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id D03E543C36	for <zzzz@localhost>; Thu, 22 Aug 2002 07:36:16 

In [172]:
def get_email_structure(email):
    if isinstance(email, str):
        return email

    payload = email.get_payload()

    if isinstance(payload, list):
        lista = []
        for sub_email in payload:
            lista.append(get_email_structure(sub_email))

        return "Estruturas do email:", lista

    else:

        return email.get_content_type()


emailsNormaisMultipart = get_email_structure(emailsNormais[634])

In [173]:
emailsNormaisMultipart

('Estruturas do email:', ['text/plain', 'text/html'])

In [174]:

for email in emailsNormais[634].walk():
    print(email)

Return-Path: <fork-admin@xent.com>
Delivered-To: yyyy@localhost.example.com
Received: from localhost (jalapeno [127.0.0.1])
	by jmason.org (Postfix) with ESMTP id C2C2316F16
	for <jm@localhost>; Fri, 13 Sep 2002 13:37:26 +0100 (IST)
Received: from jalapeno [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for jm@localhost (single-drop); Fri, 13 Sep 2002 13:37:26 +0100 (IST)
Received: from xent.com ([64.161.22.236]) by dogma.slashnull.org
    (8.11.6/8.11.6) with ESMTP id g8CNsQC20509 for <jm@jmason.org>;
    Fri, 13 Sep 2002 00:54:27 +0100
Received: from lair.xent.com (localhost [127.0.0.1]) by xent.com (Postfix)
    with ESMTP id A31B02940D3; Thu, 12 Sep 2002 16:51:04 -0700 (PDT)
Delivered-To: fork@example.com
Received: from web13003.mail.yahoo.com (web13003.mail.yahoo.com
    [216.136.174.13]) by xent.com (Postfix) with SMTP id 5696629409A for
    <fork@xent.com>; Thu, 12 Sep 2002 16:50:13 -0700 (PDT)
Message-Id: <20020912235313.4003.qmail@web13003.mail.yahoo.com>
Received: from

In [175]:
def html_to_text(email):
    try:
        soup = BeautifulSoup(email.get_content(), 'html.parser')
        return soup.text.replace('\n\n', '')
    except:
        return "empty"


def email_to_text(email):
    for emailParts in email.walk():
        if emailParts.get_content_type() not in ['text/plain', 'text/html']:
            continue

        try:
            emailPartsContent = emailParts.get_content()
        except:
            emailPartsContent = str(emailParts.get_payload())

        if emailParts.get_content_type() == 'text/plain':
            return emailPartsContent
        else:
            return html_to_text(emailParts)


'''def email_to_text(email):
    struct = get_email_structure(email)
    for part in email.walk():
        partContentType = part.get_content_type()
        if partContentType not in ['text/plain','text/html']:
            continue
        try:
            partContent = part.get_content()
        except: # in case of encoding issues
            partContent = str(part.get_payload())
        if partContentType == 'text/plain':
            return partContent
        else:
            return html_to_text(part)'''


"def email_to_text(email):\n    struct = get_email_structure(email)\n    for part in email.walk():\n        partContentType = part.get_content_type()\n        if partContentType not in ['text/plain','text/html']:\n            continue\n        try:\n            partContent = part.get_content()\n        except: # in case of encoding issues\n            partContent = str(part.get_payload())\n        if partContentType == 'text/plain':\n            return partContent\n        else:\n            return html_to_text(part)"

In [176]:
email_to_text(emailsNormais[0])

'    Date:        Wed, 21 Aug 2002 10:54:46 -0500\n    From:        Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>\n    Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>\n\n\n  | I can\'t reproduce this error.\n\nFor me it is very repeatable... (like every time, without fail).\n\nThis is the debug log of the pick happening ...\n\n18:19:03 Pick_It {exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace} {4852-4852 -sequence mercury}\n18:19:03 exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace 4852-4852 -sequence mercury\n18:19:04 Ftoc_PickMsgs {{1 hit}}\n18:19:04 Marking 1 hits\n18:19:04 tkerror: syntax error in expression "int ...\n\nNote, if I run the pick command by hand ...\n\ndelta$ pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace  4852-4852 -sequence mercury\n1 hit\n\nThat\'s where the "1 hit" comes from (obviously).  The version of nmh I\'m\nusing is ...\n\ndelta$ pick -version\npick -- nmh-1.0.4 [compiled on fuchsia.c

In [177]:
def removePunctuation(text):
    ponctuation = list(punctuation)

    for i in ponctuation:
        text = text.replace(i, " ")

    return text


def wordsPreProcessing(email):
    if email is None:
        return 'empty'

    newText = ""

    stopWords = set(stopwords.words('english') + list(STOPWORDS))
    lemmatizer = WordNetLemmatizer()
    wordTokenize = RegexpTokenizer(r'\w+')

    for text in wordTokenize.tokenize(email.lower()):
        text = removePunctuation(text)
        if text not in stopWords and not text.isdigit():
            newText += lemmatizer.lemmatize(text) + " "

    return newText

In [178]:
def bagOfWord(emails):
    wordCount = {}

    for email in emails:
        for i in email.split():
            if i not in wordCount.keys():
                wordCount[i] = 1
            else:
                wordCount[i] += 1

    return wordCount

In [179]:
emailsText = []

for email in emails:
    emailsText.append(wordsPreProcessing
                      (email_to_text(email)))

print(emailsText)

['date wed aug chris garrigues cwg dated 06fa6d deepeddy com message id tmda deepeddy vircio com reproduce error repeatable like time fail debug log pick happening pick it exec pick inbox list lbrace lbrace subject ftp rbrace rbrace sequence mercury exec pick inbox list lbrace lbrace subject ftp rbrace rbrace sequence mercury ftoc pickmsgs hit marking hit tkerror syntax error expression int note run pick command hand delta pick inbox list lbrace lbrace subject ftp rbrace rbrace sequence mercury hit hit come obviously version nmh delta pick version pick nmh compiled fuchsia c mu oz au sun mar ict relevant mh profile delta mhparam pick seq sel list pick command work sequence actually explicit command line search popup come mh profile created kre p version code form day ago able reach cv repository today local routing issue think                                                 exmh worker mailing list exmh worker redhat com http listman redhat com mailman listinfo exmh worker ', 'martin p

In [180]:
wordCount = bagOfWord(emailsText)

wordFrequency = heapq.nlargest(100, wordCount, wordCount.get)

print(wordFrequency)

['http', 'com', 'list', 'www', 'net', 'time', 'use', 'rpm', 'message', 'people', 'email', 'like', 'user', 'mail', 'new', 'free', 'linux', 'mailing', 'spamassassin', 'listinfo', 'e', 'wrote', 'year', 'work', 'date', 'html', 'file', 'exmh', 'click', 'world', 'way', 'state', 'said', 'freshrpms', 'company', 'org', 'spam', 'think', 'url', 'want', 'c', 'group', 'right', 'money', 'need', 'mailman', 'r', 'information', 'know', 'business', 'problem', 'line', 'sourceforge', 'day', 'thing', 'service', 'perl', 'address', 'good', 'report', 'razor', 'change', 'home', 'talk', 'technology', 'order', 'internet', 'government', 'send', 'run', 'redhat', 'help', 'package', 'software', 'mv', 'data', 'web', 'network', 'best', 'subject', 'program', 'site', 'yahoo', 'link', 'look', 'fork', 'set', 'check', 'number', 'news', 'sep', 'sf', 'l', 'xml', 'old', 'phone', 'life', 'server', 'today', 'country']


In [181]:
countVectorizer = CountVectorizer(analyzer="word")
tfidfVectorizer = TfidfVectorizer(analyzer="word")

countTransform = countVectorizer.fit_transform(emailsText)
tfidfTransform = tfidfVectorizer.fit_transform(emailsText)

print(countTransform)
print()
print(tfidfTransform)

  (0, 8446)	1
  (0, 31394)	1
  (0, 4119)	1
  (0, 6667)	1
  (0, 12753)	1
  (0, 8241)	1
  (0, 8447)	1
  (0, 125)	1
  (0, 8645)	2
  (0, 7126)	4
  (0, 18745)	1
  (0, 14747)	1
  (0, 29019)	1
  (0, 30855)	1
  (0, 24559)	1
  (0, 10916)	2
  (0, 24506)	1
  (0, 17541)	1
  (0, 28931)	1
  (0, 11524)	1
  (0, 8557)	1
  (0, 17788)	1
  (0, 22102)	10
  (0, 13765)	1
  (0, 15885)	1
  :	:
  (3051, 19492)	1
  (3051, 32589)	1
  (3051, 12496)	1
  (3051, 25679)	1
  (3051, 21450)	1
  (3051, 4042)	1
  (3051, 25724)	1
  (3051, 22518)	1
  (3051, 7014)	1
  (3051, 25570)	1
  (3051, 21391)	1
  (3051, 4996)	1
  (3051, 13056)	1
  (3051, 18033)	1
  (3051, 9220)	1
  (3051, 19346)	1
  (3051, 7774)	1
  (3051, 23231)	1
  (3051, 3694)	1
  (3051, 6142)	1
  (3051, 30602)	1
  (3051, 5429)	1
  (3051, 32022)	1
  (3051, 23213)	3
  (3051, 5210)	1

  (0, 17652)	0.020240486677902153
  (0, 18177)	0.023376566212732656
  (0, 17654)	0.037017937597750114
  (0, 14506)	0.011465093983824429
  (0, 24146)	0.06679878390890802
  (0, 18172)	0.02

In [182]:
countLabels = countVectorizer.get_feature_names_out()
tfidfLabels = tfidfVectorizer.get_feature_names_out()
countLabels

array(['00', '000', '0001zd', ..., '활황을', '훨씬', '힘입어'], dtype=object)

In [183]:
tfidfLabels

array(['00', '000', '0001zd', ..., '활황을', '훨씬', '힘입어'], dtype=object)

In [184]:
dfCountArray = pd.DataFrame(data=countTransform.toarray(), index=emailsNome, columns=countLabels)
dfTfidfArray = pd.DataFrame(data=tfidfTransform.toarray(), index=emailsNome, columns=tfidfLabels)

In [185]:
dfCountArray

,00,000,0001zd,0004es,0007b1d4,00085k,000e,000miles,000s,000th,...,해보자,허락없이,헤어디자이너,현재,호황을,홈쇼핑의,확실한,활황을,훨씬,힘입어
0001.ea7e79d3153e7469e7a9c3e0af6a357e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0002.b3120c4bcbf3101e661161ee7efcb8bf,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0003.acfc5ad94bbd27118a0d8685d18c89dd,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0004.e8d5727378ddde5c3be181df593f1712,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0005.8c3b9e9c0f3f183ddaf7592a11b99957,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0496.e995facc5d5bc6d3f3a8537d35e0432a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0497.9307bd1fb5347f9fbf1f54ebe95b1d20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0498.863566df8e5f17f979edca79d1e87187,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0499.4a17fbd7fe71705e09b4dd2e24d802dd,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [186]:
dfTfidfArray

,00,000,0001zd,0004es,0007b1d4,00085k,000e,000miles,000s,000th,...,해보자,허락없이,헤어디자이너,현재,호황을,홈쇼핑의,확실한,활황을,훨씬,힘입어
0001.ea7e79d3153e7469e7a9c3e0af6a357e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002.b3120c4bcbf3101e661161ee7efcb8bf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0003.acfc5ad94bbd27118a0d8685d18c89dd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0004.e8d5727378ddde5c3be181df593f1712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0005.8c3b9e9c0f3f183ddaf7592a11b99957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0496.e995facc5d5bc6d3f3a8537d35e0432a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0497.9307bd1fb5347f9fbf1f54ebe95b1d20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0498.863566df8e5f17f979edca79d1e87187,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0499.4a17fbd7fe71705e09b4dd2e24d802dd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
